# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from time import time

In [2]:
try:
    data = pd.read_csv('C:/Users/Пользователь/OneDrive/Документы/Python/autos.csv')
except:
    data = pd.read_csv('/datasets/autos.csv')
    
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

Начнем с проверки на дубликаты.

In [3]:
data.duplicated().sum()

4

In [4]:
data = data.drop_duplicates().reset_index(drop=True)

Теперь посмотрим на пропуски.

In [5]:
data.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

Заменим пропуски на заначение 'unknown'.

In [6]:
data["VehicleType"] = data["VehicleType"].fillna('unknown')
data["Gearbox"] = data["Gearbox"].fillna('unknown')
data["Model"] = data["Model"].fillna('unknown')
data["FuelType"] = data["FuelType"].fillna('unknown')
data["Repaired"] = data["Repaired"].fillna('unknown')

In [7]:
data.isna().sum()

DateCrawled          0
Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Kilometer            0
RegistrationMonth    0
FuelType             0
Brand                0
Repaired             0
DateCreated          0
NumberOfPictures     0
PostalCode           0
LastSeen             0
dtype: int64

In [8]:
data.columns = data.columns.str.lower()# приведем названия столбцов в нижний регистр.

Удалим столбцы, которые не несут важной информации для нашей модели.

In [9]:
data = data.drop(["datecrawled","datecreated","lastseen","numberofpictures","postalcode"],axis = 1)

In [10]:
data.head()

,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,repaired
0,480,unknown,1993,manual,0,golf,150000,0,petrol,volkswagen,unknown
1,18300,coupe,2011,manual,190,unknown,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,unknown
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no


Проверим датасат на наличие аномалий.

In [11]:
data.price.value_counts()

0        10772
500       5670
1500      5394
1000      4648
1200      4594
         ...  
1368         1
233          1
11080        1
16340        1
10985        1
Name: price, Length: 3731, dtype: int64

In [12]:
data = data.loc[data['price'] > 0]# Удалим значения с нулевой стоимостью.

In [13]:
data.vehicletype.unique()

array(['unknown', 'coupe', 'suv', 'small', 'sedan', 'convertible', 'bus',
       'wagon', 'other'], dtype=object)

In [14]:
data.registrationyear.unique()

array([1993, 2011, 2004, 2001, 2008, 1995, 2014, 1998, 2005, 1910, 2016,
       2007, 2009, 2002, 2018, 1997, 1990, 2017, 1981, 2003, 1994, 1991,
       1984, 2006, 1999, 2012, 2010, 2000, 1992, 2013, 1996, 1985, 1989,
       2015, 1982, 1976, 1983, 1973, 1969, 1971, 1987, 1986, 1988, 1980,
       1970, 1965, 1945, 1925, 1974, 1979, 1955, 1978, 1972, 1968, 1977,
       1961, 1966, 1975, 1963, 1964, 1960, 5000, 1958, 1967, 1959, 1956,
       3200, 1000, 1941, 9999, 8888, 1500, 2200, 4100, 1962, 1929, 1957,
       1940, 3000, 2066, 1949, 2019, 1937, 1951, 1800, 1953, 1954, 1234,
       8000, 5300, 9000, 2900, 6000, 5900, 5911, 1400, 1950, 4000, 1948,
       1952, 8500, 1932, 1255, 3700, 3800, 4800, 1942, 7000, 1935, 1933,
       1936, 6500, 1923, 2290, 1930, 1001, 9450, 1944, 2500, 1943, 1934,
       1938, 1928, 5555, 5600, 1600, 1111, 2222, 1039, 1300, 2800, 1931,
       4500, 1602, 7800, 1947, 1927, 7100, 8200, 1946], dtype=int64)

С годом регистрации автомобиля есть явные аномалии. Ограничим по годам за последние 30 лет. с 1986 по 2016.

In [15]:
data = data.loc[(data['registrationyear'] <= 2016) & (data['registrationyear'] >= 1986)]

In [16]:
data.gearbox.unique()

array(['manual', 'auto', 'unknown'], dtype=object)

In [17]:
data.power.value_counts()

0        31381
75       21961
60       14598
150      13693
101      12373
         ...  
519          1
2461         1
6006         1
10520        1
1241         1
Name: power, Length: 678, dtype: int64

Ограничим мощность мотора от 1 до 1000 л.с.

In [18]:
data = data.loc[(data['power'] <= 1000) & (data['power'] >= 1)]

In [19]:
data.kilometer.unique()

array([125000, 150000,  90000,  30000,  70000, 100000,  60000,  20000,
        80000,   5000,  50000,  40000,  10000], dtype=int64)

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292518 entries, 1 to 354364
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   price              292518 non-null  int64 
 1   vehicletype        292518 non-null  object
 2   registrationyear   292518 non-null  int64 
 3   gearbox            292518 non-null  object
 4   power              292518 non-null  int64 
 5   model              292518 non-null  object
 6   kilometer          292518 non-null  int64 
 7   registrationmonth  292518 non-null  int64 
 8   fueltype           292518 non-null  object
 9   brand              292518 non-null  object
 10  repaired           292518 non-null  object
dtypes: int64(5), object(6)
memory usage: 26.8+ MB


После обработки данных для коректной работы были удалены несколько столбцов с ненужной для нашей модели информацией и дубликаты. Пропуски в столбцах заменены на значение 'unknown'. Так же удалены записи с нулевой стоимостью, записи с годом регистрации автомобиля не попадающие во временной  промежуток с 1986 по 2016г. и записи с мощностью мотора меньше 1 и больше 1000.

## Обучение моделей

Перейдем к обучению моделей. Первым делом выделим целевой признак.

In [21]:
features=data.drop(['price'],axis=1)
target=data['price']

In [22]:
col=['vehicletype', 'gearbox','model',
       'fueltype', 'brand', 'repaired']

Разделим датасет на выборки.

In [23]:
features_train, features_valid_test, target_train, target_valid_test  = train_test_split(features,target, test_size=0.4, random_state=254)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid_test, target_valid_test, test_size=0.5, random_state=12345)

Копируем данные для кодировки

In [24]:
features_train_oe = features_train.copy()
features_valid_oe = features_valid.copy()
features_test_oe = features_test.copy()

In [25]:
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)# Задаём Энкодеру стратегию обработки неизвестных категорий.
encoder.fit(features_train[col])                                   # Обучаем Энкодер
features_train_oe[col] = encoder.transform(features_train_oe[col]) # Кодируем train
features_valid_oe[col] = encoder.transform(features_valid_oe[col]) # Кодируем valid
features_test_oe[col] = encoder.transform(features_test_oe[col])   # Кодируем test
features_test_oe.head()                                            # Смотрим

,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,repaired
24029,4.0,2000,1.0,90,37.0,150000,10,6.0,23.0,0.0
199826,5.0,1997,1.0,60,83.0,150000,2,6.0,24.0,0.0
164311,8.0,2000,1.0,118,11.0,150000,10,6.0,2.0,0.0
283674,5.0,2001,1.0,60,173.0,70000,5,6.0,38.0,1.0
267170,8.0,2006,1.0,140,29.0,150000,9,2.0,1.0,0.0


Начнем с модели DecisionTreeRegressor.

In [26]:
for depth in range(3, 22, 3):
    model = DecisionTreeRegressor(random_state=12345, max_depth = depth)
    model.fit(features_train_oe, target_train)
    predictions = model.predict(features_valid_oe)
    rmse = mean_squared_error(target_valid, predictions)**0.5
    print('Глубина:', depth)
    print('RMSE для DecisionTreeRegressor:', rmse)
    print('')

Глубина: 3
RMSE для DecisionTreeRegressor: 2863.2622408491043

Глубина: 6
RMSE для DecisionTreeRegressor: 2200.254676980122

Глубина: 9
RMSE для DecisionTreeRegressor: 1951.5520696143528

Глубина: 12
RMSE для DecisionTreeRegressor: 1825.1488627510214

Глубина: 15
RMSE для DecisionTreeRegressor: 1828.980214026706

Глубина: 18
RMSE для DecisionTreeRegressor: 1871.3380980418428

Глубина: 21
RMSE для DecisionTreeRegressor: 1930.129725568479



Лучший показатель у модели DecisionTreeRegressor на глубине 12. и RMSE 1825.1488627510214


Далее посмотрим на RandomForestRegressor.

In [27]:
for depth in range(1,6):
    for est in range(3,22,3):
        model = RandomForestRegressor(max_depth=depth, n_estimators=est, random_state=1234)
        model.fit(features_train_oe, target_train)
        predictions = model.predict(features_valid_oe)
        rmse = mean_squared_error(target_valid, predictions)**0.5
        print('Количество деревьев:', est, "Глубина",depth)
        print('RMSE для RandomForestRegressor:', rmse)
        print('')

Количество деревьев: 3 Глубина 1
RMSE для RandomForestRegressor: 3615.9276075344374

Количество деревьев: 6 Глубина 1
RMSE для RandomForestRegressor: 3615.9532291467

Количество деревьев: 9 Глубина 1
RMSE для RandomForestRegressor: 3615.9542835935295

Количество деревьев: 12 Глубина 1
RMSE для RandomForestRegressor: 3615.950526539263

Количество деревьев: 15 Глубина 1
RMSE для RandomForestRegressor: 3615.9377955818586

Количество деревьев: 18 Глубина 1
RMSE для RandomForestRegressor: 3615.9394095974662

Количество деревьев: 21 Глубина 1
RMSE для RandomForestRegressor: 3615.9468780511534

Количество деревьев: 3 Глубина 2
RMSE для RandomForestRegressor: 3140.0699612342437

Количество деревьев: 6 Глубина 2
RMSE для RandomForestRegressor: 3139.7927900605423

Количество деревьев: 9 Глубина 2
RMSE для RandomForestRegressor: 3140.01279128729

Количество деревьев: 12 Глубина 2
RMSE для RandomForestRegressor: 3139.838525616604

Количество деревьев: 15 Глубина 2
RMSE для RandomForestRegressor: 3

Лучший показатель RMSE у модели RandomForestRegressor равен 2298.541594925894, что хуже, чем у DecisionTreeRegressor.

Далее посмотрим на LGBMRegressor.

In [28]:
for est in [100, 500, 1000]:
    for num in [25, 400, 15]:
        model = lgb.LGBMRegressor(n_estimators = est, num_leaves = num, random_state=12345)
        model.fit(features_train_oe, target_train)
        predictions= model.predict(features_valid_oe)
        rmse = mean_squared_error(target_valid, predictions)**0.5
        print('Количество деревьев:', est, num)
        print('RMSE для LGBMRegressor:', rmse)
        print('')

Количество деревьев: 100 25
RMSE для LGBMRegressor: 1642.6151759206161

Количество деревьев: 100 400
RMSE для LGBMRegressor: 1464.0072258613543

Количество деревьев: 100 15
RMSE для LGBMRegressor: 1699.4026205883454

Количество деревьев: 500 25
RMSE для LGBMRegressor: 1510.8533436413638

Количество деревьев: 500 400
RMSE для LGBMRegressor: 1446.5234388913013

Количество деревьев: 500 15
RMSE для LGBMRegressor: 1553.039446344039

Количество деревьев: 1000 25
RMSE для LGBMRegressor: 1479.842296693585

Количество деревьев: 1000 400
RMSE для LGBMRegressor: 1461.448136699782

Количество деревьев: 1000 15
RMSE для LGBMRegressor: 1508.6833687953942



А вот и победитель! RMSE 1446.5234388913013

Сравнив три модели можно выделить наиболее эффективную. В нашем случае это LGBMRegressor.

## Анализ моделей

Теперь посчитаем отдельно время затраченное на обучение и предсказания каждой модели.

DecisionTreeRegressor.

In [29]:
start = time()
model1 = DecisionTreeRegressor(random_state=12345, max_depth=12)
model1.fit(features_train_oe, target_train)
end = time()
t1 = round(end - start, 3)
print('Время обучения:', t1, 'сек.')

Время обучения: 1.018 сек.


In [30]:
start = time()
predictions = model1.predict(features_valid_oe)
end = time()
t2 = round(end - start, 3)
rmse_1 = mean_squared_error(target_valid, predictions)**0.5
print('Время предсказания:', t2, 'сек.')
print('RMSE для DecisionTreeRegressor:', rmse_1)

Время предсказания: 0.031 сек.
RMSE для DecisionTreeRegressor: 1825.1488627510214


RandomForestRegressor

In [31]:
start = time()
model2 = RandomForestRegressor(random_state=12345, max_depth=4, n_estimators=15)
model2.fit(features_train_oe, target_train)
end = time()
t3 = round(end - start, 3)
print('Время обучения:', t3, 'сек.')

Время обучения: 3.941 сек.


In [32]:
start = time()
predictions = model2.predict(features_valid_oe)
end = time()
t4 = round(end - start, 3)
rmse_2 = mean_squared_error(target_valid, predictions)**0.5
print('Время предсказания:', t4, 'сек.')
print('RMSE для RandomForestRegressor:', rmse_2)

Время предсказания: 0.093 сек.
RMSE для RandomForestRegressor: 2488.3153396652447


LGBMRegressor

In [33]:
start = time()
model3 = lgb.LGBMRegressor(random_state=12345,
                          n_estimators=500,
                          num_leaves=400)
model3.fit(features_train_oe, target_train)
end = time()
t5 = round(end - start, 3)
print('Время обучения:', t5, 'сек.')

Время обучения: 12.791 сек.


In [34]:
start = time()
predictions = model3.predict(features_valid_oe)
end = time()
t6 = round(end - start, 3)
rmse_3 = mean_squared_error(target_valid, predictions)**0.5
print('Время предсказания:', t6, 'сек.')
print('RMSE для LGBMRegressor:', rmse_3)

Время предсказания: 1.655 сек.
RMSE для LGBMRegressor: 1446.5234388913013


In [35]:
analysis = [['DecisionTreeRegressor', rmse_1, t1, t2],
            ['RandomForestRegressor', rmse_2, t3, t4],
            ['LGBMRegressor', rmse_3, t5, t6]]
columns = ['Модель','RMSE', 'Время обучения', 'Время предсказания']
analysis_data = pd.DataFrame(data=analysis, columns=columns)
analysis_data.sort_values(by='RMSE', axis=0).reset_index(drop=True)

,Модель,RMSE,Время обучения,Время предсказания
0,LGBMRegressor,1446.523439,12.791,1.655
1,DecisionTreeRegressor,1825.148863,1.018,0.031
2,RandomForestRegressor,2488.315340,3.941,0.093


Чтож, хорошо видно, что модель LGBMRegressor затрачивает зничительно больше времени на обучение и предсказание. Но при этом эта потеря времени оправдана лучшим показателем RMSE.

## Тестирование лучшей модели

Проверим результаты модели LGBMRegressor на тестовой выборке.

In [36]:
predictions = model3.predict(features_test_oe)
rmse_4 = mean_squared_error(target_test, predictions)**0.5
print('RMSE для LGBMRegressor на тестовой выборке:', rmse_4)

RMSE для LGBMRegressor на тестовой выборке: 1466.4245663680522


Отлично, расхождения минимальны. Тестирование прошло удачно.

## Общие выводы

Была проведена предварительная обработка данных по выявлению пропусков, дубликатов и удалению ненужной нам информации.

На примере построения трех моделей был выявлен яыный фаворит с наилучшими показателями метрике RMSE. Это модель LGBMRegressor.
И несмотря на большее затраченное время предсказания именно ее лучше рекомендовать заказчику для получения наиболее точной цены на авто.
Возможно имеет смысл использовать две модели на разных этапах регистрации. Предварительно использовать более быструю, но менее точную модель для привлечения внимания пользователя. А потом при более полном заполнеении информации об авто уже использовать более долгую, но точную модель.